In [1]:
import pickle 
import numpy as np

In [2]:
with open('train_qa.txt','rb') as f:
    train_data=pickle.load(f)

In [3]:
with open('train_qa.txt','rb') as f:
    test_data=pickle.load(f)

In [4]:
type(test_data)

list

In [5]:
len(train_data)

10000

In [6]:
len(test_data)

10000

In [8]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [11]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [12]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [14]:
''.join(train_data[0][2])

'no'

In [15]:
all_data=test_data+train_data

In [16]:
len(all_data)

20000

In [18]:
vocab=set()
for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab=vocab.union(set(question))

In [19]:
vocab.add('no')

In [20]:
vocab.add('yes')

In [21]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [22]:
vocab_len=len(vocab)+1

In [23]:
vocab_len

38

In [26]:
all_story_lens=[len(data[0]) for data in all_data]

In [28]:
max_story_len=max(all_story_lens)

In [30]:
max_question_len=max([len(data[1]) for data in all_data])

In [31]:
max_question_len

6

In [32]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [36]:
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [37]:
tokenizer.word_index

{'milk': 1,
 'garden': 2,
 'dropped': 3,
 'sandra': 4,
 'office': 5,
 'there': 6,
 'moved': 7,
 '.': 8,
 'got': 9,
 'mary': 10,
 'bedroom': 11,
 'daniel': 12,
 'no': 13,
 'back': 14,
 'up': 15,
 'grabbed': 16,
 'bathroom': 17,
 'kitchen': 18,
 'left': 19,
 'put': 20,
 'yes': 21,
 'apple': 22,
 'went': 23,
 'in': 24,
 'to': 25,
 'down': 26,
 'took': 27,
 'hallway': 28,
 'picked': 29,
 'the': 30,
 'football': 31,
 'john': 32,
 '?': 33,
 'travelled': 34,
 'discarded': 35,
 'is': 36,
 'journeyed': 37}

In [43]:
train_story_text=[]

In [44]:
train_question_text=[]

In [45]:
train_answers=[]

In [47]:
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [48]:
train_story_seq=tokenizer.texts_to_sequences(train_story_text)

In [49]:
len(train_story_text)

10000

In [50]:
train_story_seq

[[10, 7, 25, 30, 17, 8, 4, 37, 25, 30, 11, 8],
 [10,
  7,
  25,
  30,
  17,
  8,
  4,
  37,
  25,
  30,
  11,
  8,
  10,
  23,
  14,
  25,
  30,
  11,
  8,
  12,
  23,
  14,
  25,
  30,
  28,
  8],
 [10,
  7,
  25,
  30,
  17,
  8,
  4,
  37,
  25,
  30,
  11,
  8,
  10,
  23,
  14,
  25,
  30,
  11,
  8,
  12,
  23,
  14,
  25,
  30,
  28,
  8,
  4,
  23,
  25,
  30,
  18,
  8,
  12,
  23,
  14,
  25,
  30,
  17,
  8],
 [10,
  7,
  25,
  30,
  17,
  8,
  4,
  37,
  25,
  30,
  11,
  8,
  10,
  23,
  14,
  25,
  30,
  11,
  8,
  12,
  23,
  14,
  25,
  30,
  28,
  8,
  4,
  23,
  25,
  30,
  18,
  8,
  12,
  23,
  14,
  25,
  30,
  17,
  8,
  12,
  29,
  15,
  30,
  31,
  6,
  8,
  12,
  23,
  25,
  30,
  11,
  8],
 [10,
  7,
  25,
  30,
  17,
  8,
  4,
  37,
  25,
  30,
  11,
  8,
  10,
  23,
  14,
  25,
  30,
  11,
  8,
  12,
  23,
  14,
  25,
  30,
  28,
  8,
  4,
  23,
  25,
  30,
  18,
  8,
  12,
  23,
  14,
  25,
  30,
  17,
  8,
  12,
  29,
  15,
  30,
  31,
  6,
  8,
  12,
  23

In [58]:
def vectorize_story(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    X=[]
    Xq=[]
    Y=[]
    
    for story,query,answer in data:
        x=[word_index[word.lower()] for word in story]
        xq=[word_index[word.lower()] for word in query]
        
        y=np.zeros(len(word_index)+1)
        
        y[word_index[answer]]==1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))
        

In [66]:
inputs_train,queries_train,answer_train=vectorize_story(train_data)

In [67]:
inputs_test,queries_test,answer_test=vectorize_story(train_data)

In [68]:
inputs_train

array([[ 0,  0,  0, ..., 30, 11,  8],
       [ 0,  0,  0, ..., 30, 28,  8],
       [ 0,  0,  0, ..., 30, 17,  8],
       ...,
       [ 0,  0,  0, ..., 30, 11,  8],
       [ 0,  0,  0, ...,  1,  6,  8],
       [ 0,  0,  0, ..., 22,  6,  8]], dtype=int32)

In [69]:
answer_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [70]:
tokenizer.word_index['yes']

21

In [71]:
tokenizer.word_index['no']

13

In [72]:
sum(answer_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [73]:
from keras.models import Sequential,Model
from keras.layers.embeddings import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate

In [74]:
from keras.layers import LSTM

In [75]:
input_sequence=Input((max_story_len,))
question=Input((max_question_len,))

In [76]:
vocab_size=len(vocab)+1

In [78]:
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [79]:
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

In [80]:
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

In [99]:

input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [100]:
match=dot([input_encoded_m,question_encoded],axes=(2,2))

In [101]:
match=Activation('softmax')(match)

In [102]:
response =add([match,input_encoded_c])
response=Permute((2,1))(response)

In [103]:
answer=concatenate([response,question_encoded])

In [104]:
answer

<tf.Tensor 'concatenate_1/Identity:0' shape=(None, 6, 220) dtype=float32>

In [105]:
answer=LSTM(32)(answer)

In [106]:
answer=Dense(vocab_size)(answer)

In [107]:
answer=Activation('softmax')(answer)

In [109]:
model=Model([input_sequence,question],answer)

In [111]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [112]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 156)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
____________________________________________________________________________________________

In [119]:
# train
history = model.fit([inputs_train, queries_train], answer_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answer_test))

Epoch 1/120
313/313 [==============================] - 6s 18ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/120
313/313 [==============================] - 6s 19ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/120
313/313 [==============================] - 5s 17ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/120
313/313 [==============================] - 7s 21ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/120
313/313 [==============================] - 5s 17ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/120
313/313 [==============================] - 6s 19ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/120
313/313 [===================

Epoch 52/120
313/313 [==============================] - 5s 16ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 53/120
313/313 [==============================] - 6s 18ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 54/120
313/313 [==============================] - 7s 23ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 55/120
313/313 [==============================] - 5s 17ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 56/120
313/313 [==============================] - 7s 23ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 57/120
313/313 [==============================] - 6s 19ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 58/120
313/313 [============

Epoch 103/120
313/313 [==============================] - 8s 27ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 104/120
313/313 [==============================] - 9s 29ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 105/120
313/313 [==============================] - 8s 26ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 106/120
313/313 [==============================] - 8s 26ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 107/120
313/313 [==============================] - 8s 26ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 108/120
313/313 [==============================] - 8s 27ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 109/120
313/313 [=====